# Model Validation Pipeline

Identified best parameters in ModelTraining_Pipeline

This pipeline is only used to get the model's performance using training data against validation data.

In [ ]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 7.07 s (started: 2023-01-16 11:35:35 +00:00)


In [ ]:
# Basic Libraries

import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from datetime import datetime

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 20.5 s (started: 2023-01-16 11:35:42 +00:00)


In [ ]:
# Data Source

df = pd.read_csv("/content/drive/MyDrive/cleaned_gee_data.csv")
df = df.drop(columns = ['Unnamed: 0'], axis=1) # Remove index
df.head()

,LATITUDE,LONGITUDE,ACQ_DATE,ACQ_TIME,OPEN_TIME,CLOSE_TIME,FIRE_OCCURRED,CO_MOL/M2,SO2_MOL/M2,NO2_MOL/M2,O3_MOL/M2,LOCATION,INSTRUMENT,DRY_SEASON
0,-5.466232,-0.176027,-1.866392,0.634294,0.506405,0.526945,0,-0.024223,-0.474440,-1.152277,-0.511001,-1.159086,0,1
1,-5.466232,-0.176027,-1.866392,0.634294,0.506405,0.526945,0,0.113599,-0.474440,-1.152277,-0.511001,-1.159086,0,1
2,-5.466232,-0.176027,-1.866392,0.634294,0.506405,0.526945,0,-0.024223,-0.474440,-1.361255,-0.511001,-1.159086,0,1
3,-5.466232,-0.176027,-1.866392,0.634294,0.506405,0.526945,0,0.113599,-0.474440,-1.361255,-0.511001,-1.159086,0,1
4,-5.433352,-0.197441,-1.723773,0.634294,2.286080,1.793843,0,-0.967684,0.339667,-1.251770,0.426114,-1.159086,0,1


time: 1.7 s (started: 2023-01-16 11:36:02 +00:00)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171893 entries, 0 to 171892
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   LATITUDE       171893 non-null  float64
 1   LONGITUDE      171893 non-null  float64
 2   ACQ_DATE       171893 non-null  float64
 3   ACQ_TIME       171893 non-null  float64
 4   OPEN_TIME      171893 non-null  float64
 5   CLOSE_TIME     171893 non-null  float64
 6   FIRE_OCCURRED  171893 non-null  int64  
 7   CO_MOL/M2      171893 non-null  float64
 8   SO2_MOL/M2     171893 non-null  float64
 9   NO2_MOL/M2     171893 non-null  float64
 10  O3_MOL/M2      171893 non-null  float64
 11  LOCATION       171893 non-null  float64
 12  INSTRUMENT     171893 non-null  int64  
 13  DRY_SEASON     171893 non-null  int64  
dtypes: float64(11), int64(3)
memory usage: 18.4 MB
time: 24.9 ms (started: 2023-01-16 11:36:04 +00:00)


In [ ]:
display(df['FIRE_OCCURRED'].value_counts())

0    170544
1      1349
Name: FIRE_OCCURRED, dtype: int64

time: 7.88 ms (started: 2023-01-16 11:36:04 +00:00)


In [ ]:
X = df.drop('FIRE_OCCURRED', axis=1)
y = df['FIRE_OCCURRED']

time: 9 ms (started: 2023-01-16 11:36:04 +00:00)


In [ ]:
# Training, Validation, Testing Split

from sklearn.model_selection import train_test_split

# 80:10:10

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=10, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=1/9, random_state=10, shuffle=True)

Original = [X_train, X_val, X_test, y_train, y_val, y_test] # For reference

time: 356 ms (started: 2023-01-16 11:36:04 +00:00)


In [ ]:
if len(X_train)==len(y_train) and len(X_test) == len(y_test) and len(X_val) == len(y_val):
  print("X and y data length matching")
else:
  print("Error in data preparation pipeline")
print()
print("No. of training data = %d" % len(X_train))
print("No. of validation data = %d" % len(X_val))
print("No. of testing data = %d" % len(X_test))

X and y data length matching

No. of training data = 137513
No. of validation data = 17190
No. of testing data = 17190
time: 2.02 ms (started: 2023-01-16 11:36:04 +00:00)


In [ ]:
display(y_val.value_counts())

0    17068
1      122
Name: FIRE_OCCURRED, dtype: int64

time: 15.9 ms (started: 2023-01-16 11:36:04 +00:00)


In [ ]:
display(y_test.value_counts())

0    17059
1      131
Name: FIRE_OCCURRED, dtype: int64

time: 5.68 ms (started: 2023-01-16 11:36:04 +00:00)


In [ ]:
# SMOTE

from collections import Counter
from imblearn.over_sampling import SMOTE 

print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(random_state=10)
X_train, y_train = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_train))

Original dataset shape Counter({0: 136417, 1: 1096})
Resampled dataset shape Counter({0: 136417, 1: 136417})
time: 895 ms (started: 2023-01-16 11:36:04 +00:00)


In [ ]:
# Shuffle Data since SMOTE appended many 1s at the end
# Required for some algorithms such as ANN

from sklearn.utils import shuffle

X_train, y_train = shuffle(X_train, y_train, random_state = 10)

time: 83.8 ms (started: 2023-01-16 11:36:05 +00:00)


In [ ]:
# Evaluation Metrics

from sklearn.metrics import confusion_matrix, recall_score, f1_score, roc_auc_score, accuracy_score

def evaluation_metrics(y_true, y_pred):
  cfm = confusion_matrix(y_true, y_pred).ravel()
  acc = accuracy_score(y_true, y_pred)
  recs = recall_score(y_true, y_pred, average='binary')
  f1s = f1_score(y_true, y_pred, average='binary')
  rocs = roc_auc_score(y_true, y_pred, average='macro')
  return [cfm, acc, recs, f1s, rocs]

time: 1.32 ms (started: 2023-01-16 11:36:05 +00:00)


Confusion matrix format : [ tn , fp , fn , tp ]

In [ ]:
# Store Model Parameters and Eval

models = pd.DataFrame(columns = ['model_name', 'model', 'parameters'])
models_eval = pd.DataFrame(columns = ['model_name', 'confusion_matrix', 'accuracy', 'recall', 'f1_score', 'roc_auc_score'])

time: 8.05 ms (started: 2023-01-16 11:36:05 +00:00)


In [ ]:
# Import ML Algorithms

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost
from xgboost import XGBClassifier
import lightgbm
from lightgbm import LGBMClassifier
import tensorflow as tf
from tensorflow import keras
from sklearn.ensemble import VotingClassifier

time: 5.29 s (started: 2023-01-16 11:36:05 +00:00)


## Logistic Regression

- Library: Scikit-learn

**Best Parameters:**

{'warm_start': True,
 'solver': 'lbfgs',
 'penalty': 'none',
 'max_iter': 331,
 'dual': False,
 'C': 0}

In [ ]:
name = 'log_clf'

log_clf = LogisticRegression(penalty = 'none', 
                             warm_start = True,
                             solver = 'lbfgs',
                             max_iter = 331,
                             dual = False,
                             C = 0,
                             n_jobs = -1, 
                             random_state = 10
                             ).fit(X_train,y_train)

y_true = y_val
y_pred = log_clf.predict(X_val)
evaluation_results = evaluation_metrics(y_true, y_pred)

models = models.append({'model_name': name, 
                        'model': log_clf, 
                        'parameters': log_clf.get_params()}, 
                        ignore_index=True)

models_eval = models_eval.append({'model_name': name, 
                                  'confusion_matrix' : evaluation_results[0], 
                                  'accuracy': evaluation_results[1], 
                                  'recall' : evaluation_results[2], 
                                  'f1_score': evaluation_results[3],
                                  'roc_auc_score': evaluation_results[4]}, 
                                 ignore_index=True)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


time: 8.12 s (started: 2023-01-16 11:36:11 +00:00)


## Support Vector Machine (SVM)

- Library: Scikit-learn

**Best Parameters:**

{
'kernel': 'rbf',
'C' : '8',
'class_weight'='balanced'
}

In [ ]:
X_train_SVM = Original[0]
X_val_SVM = Original[1]
y_train_SVM = Original[3]
y_val_SVM = Original[4]

time: 5.32 ms (started: 2023-01-16 11:36:19 +00:00)


In [ ]:
# Shuffle

X_train_SVM, y_train_SVM = shuffle(X_train_SVM, y_train_SVM, random_state = 10)

time: 245 ms (started: 2023-01-16 11:36:19 +00:00)


In [ ]:
display(y_train_SVM.value_counts())

0    136417
1      1096
Name: FIRE_OCCURRED, dtype: int64

time: 37.7 ms (started: 2023-01-16 11:36:19 +00:00)


In [ ]:
name = 'svc_clf'

svc_clf = SVC(kernel = 'rbf', 
              C = 8,
              class_weight='balanced',
              random_state = 10
              ).fit(X_train_SVM,y_train_SVM)

y_true = y_val_SVM
y_pred = svc_clf.predict(X_val_SVM)
evaluation_results = evaluation_metrics(y_true, y_pred)

models = models.append({'model_name': name, 
                        'model': svc_clf, 
                        'parameters': svc_clf.get_params()}, 
                       ignore_index=True)

models_eval = models_eval.append({'model_name': name, 
                                  'confusion_matrix' : evaluation_results[0], 
                                  'accuracy': evaluation_results[1], 
                                  'recall' : evaluation_results[2], 
                                  'f1_score': evaluation_results[3],
                                  'roc_auc_score': evaluation_results[4]}, 
                                 ignore_index=True)

time: 4min 35s (started: 2023-01-16 11:36:19 +00:00)


## Naive Bayes

- Library: Scikit-learn

**Best Parameters:**

{'var_smoothing': 1e-3}

In [ ]:
name = 'bayes_clf'

bayes_clf = GaussianNB(var_smoothing = 1e-3
                       ).fit(X_train,y_train)

y_true = y_val
y_pred = bayes_clf.predict(X_val)
evaluation_results = evaluation_metrics(y_true, y_pred)

models = models.append({'model_name': name, 
                        'model': bayes_clf, 
                        'parameters': bayes_clf.get_params()}, 
                       ignore_index=True)

models_eval = models_eval.append({'model_name': name, 
                                  'confusion_matrix' : evaluation_results[0], 
                                  'accuracy': evaluation_results[1], 
                                  'recall' : evaluation_results[2], 
                                  'f1_score': evaluation_results[3],
                                  'roc_auc_score': evaluation_results[4]}, 
                                 ignore_index=True)

time: 148 ms (started: 2023-01-16 11:40:55 +00:00)


## K-Nearest Neighbor

- Library: Scikit-learn

**Best Parameters:**

{'n_neighbors': 19, 'algorithm': 'kd_tree'}

In [ ]:
name = 'neigh_clf'

neigh_clf = KNeighborsClassifier(n_neighbors = 19, 
                                 algorithm = 'kd_tree',
                                 n_jobs = -1, 
                                 ).fit(X_train,y_train)

y_true = y_val
y_pred = neigh_clf.predict(X_val)
evaluation_results = evaluation_metrics(y_true, y_pred)

models = models.append({'model_name': name, 
                        'model': neigh_clf, 
                        'parameters': neigh_clf.get_params()}, 
                       ignore_index=True)

models_eval = models_eval.append({'model_name': name, 
                                  'confusion_matrix' : evaluation_results[0], 
                                  'accuracy': evaluation_results[1], 
                                  'recall' : evaluation_results[2], 
                                  'f1_score': evaluation_results[3],
                                  'roc_auc_score': evaluation_results[4]}, 
                                 ignore_index=True)

time: 11.9 s (started: 2023-01-16 11:40:55 +00:00)


## Decision Tree

- Library: Scikit-learn

**Best Parameters:**

{'splitter': 'random',
 'min_samples_leaf': 2,
 'max_features': 11,
 'max_depth': None,
 'criterion': 'entropy'}

In [ ]:
name = 'tree_clf'

tree_clf = DecisionTreeClassifier(criterion = 'entropy', 
                                  splitter = 'random', 
                                  min_samples_leaf = 2,
                                  max_features = 11,
                                  max_depth = None,
                                  random_state = 10
                                  ).fit(X_train,y_train)

y_true = y_val
y_pred = tree_clf.predict(X_val)
evaluation_results = evaluation_metrics(y_true, y_pred)

models = models.append({'model_name': name, 
                        'model': tree_clf, 
                        'parameters': tree_clf.get_params()}, 
                       ignore_index=True)

models_eval = models_eval.append({'model_name': name, 
                                  'confusion_matrix' : evaluation_results[0], 
                                  'accuracy': evaluation_results[1], 
                                  'recall' : evaluation_results[2], 
                                  'f1_score': evaluation_results[3],
                                  'roc_auc_score': evaluation_results[4]}, 
                                 ignore_index=True)

time: 647 ms (started: 2023-01-16 11:41:07 +00:00)


## Random Forest Classifier

- Library: Scikit-learn

**Best Parameters:**

{'n_estimators': 415,
 'min_samples_split': 6,
 'min_samples_leaf': 1,
 'max_features': 4,
 'max_depth': 18}

In [ ]:
name = 'rnd_clf'

rnd_clf = RandomForestClassifier(n_estimators = 415, 
                                  min_samples_split = 6,
                                  min_samples_leaf = 1,
                                  max_features = 4,
                                  max_depth = 18, 
                                  n_jobs = -1, 
                                  random_state = 10
                                  ).fit(X_train,y_train)

y_true = y_val
y_pred = rnd_clf.predict(X_val)
evaluation_results = evaluation_metrics(y_true, y_pred)

models = models.append({'model_name': name, 
                        'model': rnd_clf, 
                        'parameters': rnd_clf.get_params()}, 
                       ignore_index=True)

models_eval = models_eval.append({'model_name': name, 
                                  'confusion_matrix' : evaluation_results[0], 
                                  'accuracy': evaluation_results[1], 
                                  'recall' : evaluation_results[2], 
                                  'f1_score': evaluation_results[3],
                                  'roc_auc_score': evaluation_results[4]}, 
                                 ignore_index=True)

time: 4min 15s (started: 2023-01-16 11:41:08 +00:00)


## Gradient Boosting Classifier

- Library: Scikit-learn

**Best Parameters:**

{'n_estimators': 1000, 'max_depth': 8, 'learning_rate': 0.1}

In [ ]:
name = 'gboost_clf'

gboost_clf = GradientBoostingClassifier(n_estimators = 1000, 
                                        learning_rate = 0.1, 
                                        max_depth = 8,
                                        random_state = 10
                                        ).fit(X_train,y_train)

y_true = y_val
y_pred = gboost_clf.predict(X_val)
evaluation_results = evaluation_metrics(y_true, y_pred)

models = models.append({'model_name': name, 
                        'model': gboost_clf, 
                        'parameters': gboost_clf.get_params()}, 
                       ignore_index=True)

models_eval = models_eval.append({'model_name': name, 
                                  'confusion_matrix' : evaluation_results[0], 
                                  'accuracy': evaluation_results[1], 
                                  'recall' : evaluation_results[2], 
                                  'f1_score': evaluation_results[3],
                                  'roc_auc_score': evaluation_results[4]}, 
                                 ignore_index=True)

time: 39min 43s (started: 2023-01-16 11:45:23 +00:00)


## XGBoost

- Library: xgboost

**Best Parameters:**

{'n_estimators': 1000,
 'min_child_weight': 7,
 'max_depth': 8,
 'learning_rate': 0.1}

In [ ]:
name = 'xgboost_clf'

xgboost_clf = XGBClassifier(n_estimators = 1000, 
                            learning_rate = 0.1,
                            max_depth = 8, 
                            min_child_weight = 7,
                            random_state = 10
                            ).fit(X_train,y_train)

y_true = y_val
y_pred = xgboost_clf.predict(X_val)
evaluation_results = evaluation_metrics(y_true, y_pred)

models = models.append({'model_name': name, 
                        'model': xgboost_clf, 
                        'parameters': xgboost_clf.get_params()}, 
                       ignore_index=True)

models_eval = models_eval.append({'model_name': name, 
                                  'confusion_matrix' : evaluation_results[0], 
                                  'accuracy': evaluation_results[1], 
                                  'recall' : evaluation_results[2], 
                                  'f1_score': evaluation_results[3],
                                  'roc_auc_score': evaluation_results[4]}, 
                                 ignore_index=True)

time: 10min 21s (started: 2023-01-16 12:25:06 +00:00)


## LightGBM

- Library: lightbgm

**Best Parameters:**

{'num_leaves': 50,
 'n_estimators': 1000,
 'min_data_in_leaf': 10,
 'max_depth': 8,
 'learning_rate': 0.05}

In [ ]:
name = 'lightgbm_clf'

lightgbm_clf = LGBMClassifier(n_estimators = 1000, 
                              learning_rate = 0.05, 
                              max_depth = 8,
                              num_leaves = 50,
                              min_data_in_leaf = 10,
                              random_state = 10
                              ).fit(X_train,y_train)

y_true = y_val
y_pred = lightgbm_clf.predict(X_val)
evaluation_results = evaluation_metrics(y_true, y_pred)

models = models.append({'model_name': name, 
                        'model': lightgbm_clf, 
                        'parameters': lightgbm_clf.get_params()}, 
                       ignore_index=True)

models_eval = models_eval.append({'model_name': name, 
                                  'confusion_matrix' : evaluation_results[0], 
                                  'accuracy': evaluation_results[1], 
                                  'recall' : evaluation_results[2], 
                                  'f1_score': evaluation_results[3],
                                  'roc_auc_score': evaluation_results[4]}, 
                                 ignore_index=True)

time: 43.6 s (started: 2023-01-16 12:35:28 +00:00)


## Artificial Neural Network

- Library: Keras, Tensorflow

**Best Parameters:**
- Batch size 15
- EPOCH 50

In [ ]:
tf.random.set_seed(10)

time: 1.44 ms (started: 2023-01-16 12:36:12 +00:00)


In [ ]:
name = 'ann_clf'

ann_clf = keras.models.Sequential([
    keras.layers.Dense(15, input_shape=(X_train.shape[1],), activation='relu'), # No bias term
    # keras.layers.Dense(10, activation='relu'), 
    keras.layers.Dense(10, activation='relu'), 
    keras.layers.Dense(1, activation='sigmoid')
])

ann_clf.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 15)                210       
                                                                 
 dense_1 (Dense)             (None, 10)                160       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 381
Trainable params: 381
Non-trainable params: 0
_________________________________________________________________
time: 427 ms (started: 2023-01-16 12:36:12 +00:00)


In [ ]:
display(y_train.value_counts())

0    136417
1    136417
Name: FIRE_OCCURRED, dtype: int64

time: 16.7 ms (started: 2023-01-16 12:36:12 +00:00)


In [ ]:
display(y_test.value_counts())

0    17059
1      131
Name: FIRE_OCCURRED, dtype: int64

time: 8.62 ms (started: 2023-01-16 12:36:12 +00:00)


In [ ]:
ann_clf.compile(optimizer = 'adam', 
                metrics=['accuracy'], 
                loss ='binary_crossentropy')

record = ann_clf.fit(
            X_train, 
            y_train, 
            validation_data = (X_val, y_val), 
            batch_size = 15, 
            epochs = 50)

Epoch 1/50
18189/18189 [==============================] - 47s 3ms/step - loss: 0.2149 - accuracy: 0.9200 - val_loss: 0.2072 - val_accuracy: 0.9083
Epoch 2/50
18189/18189 [==============================] - 48s 3ms/step - loss: 0.1674 - accuracy: 0.9405 - val_loss: 0.1730 - val_accuracy: 0.9247
Epoch 3/50
18189/18189 [==============================] - 48s 3ms/step - loss: 0.1549 - accuracy: 0.9450 - val_loss: 0.1862 - val_accuracy: 0.9226
Epoch 4/50
18189/18189 [==============================] - 47s 3ms/step - loss: 0.1494 - accuracy: 0.9470 - val_loss: 0.1797 - val_accuracy: 0.9219
Epoch 5/50
18189/18189 [==============================] - 47s 3ms/step - loss: 0.1454 - accuracy: 0.9488 - val_loss: 0.1563 - val_accuracy: 0.9337
Epoch 6/50
18189/18189 [==============================] - 48s 3ms/step - loss: 0.1419 - accuracy: 0.9500 - val_loss: 0.1608 - val_accuracy: 0.9312
Epoch 7/50
18189/18189 [==============================] - 45s 2ms/step - loss: 0.1384 - accuracy: 0.9519 - val_loss: 0

In [ ]:
y_true = y_val
prediction = ann_clf.predict(X_val)
prediction = pd.Series(prediction[:, 0])
y_pred = []

for i in range(len(prediction)):
  if prediction[i] >= 0.5:
    y_pred.append(1)
  else:
    y_pred.append(0)

evaluation_results = evaluation_metrics(y_true, y_pred)

models = models.append({'model_name': name, 
                        'model': ann_clf, 
                        'parameters': ann_clf.layers}, 
                       ignore_index=True)

models_eval = models_eval.append({'model_name': name, 
                                  'confusion_matrix' : evaluation_results[0], 
                                  'accuracy': evaluation_results[1], 
                                  'recall' : evaluation_results[2], 
                                  'f1_score': evaluation_results[3],
                                  'roc_auc_score': evaluation_results[4]}, 
                                 ignore_index=True)

538/538 [==============================] - 1s 1ms/step
time: 1.56 s (started: 2023-01-16 13:15:35 +00:00)


## Voting Classifier
- Library: Scikit-learn, Keras, Tensorflow

**Best Parameters:**

{'voting': 'hard'}

In [ ]:
display(models_eval)

,model_name,confusion_matrix,accuracy,recall,f1_score,roc_auc_score
0,log_clf,"[14046, 3022, 42, 80]",0.821757,0.655738,0.049628,0.739341
1,svc_clf,"[16317, 751, 14, 108]",0.955497,0.885246,0.220183,0.920623
2,bayes_clf,"[11475, 5593, 18, 104]",0.673589,0.852459,0.035745,0.762385
3,neigh_clf,"[16554, 514, 16, 106]",0.969168,0.868852,0.285714,0.919369
4,tree_clf,"[16930, 138, 24, 98]",0.990576,0.803279,0.547486,0.897597
5,rnd_clf,"[16838, 230, 21, 101]",0.985398,0.827869,0.445916,0.907197
6,gboost_clf,"[17017, 51, 20, 102]",0.995870,0.836066,0.741818,0.916539
7,xgboost_clf,"[17005, 63, 23, 99]",0.994997,0.811475,0.697183,0.903892
8,lightgbm_clf,"[16980, 88, 22, 100]",0.993601,0.819672,0.645161,0.907258
9,ann_clf,"[16079, 989, 14, 108]",0.941652,0.885246,0.177194,0.913651


time: 21.3 ms (started: 2023-01-16 13:15:37 +00:00)


Confusion matrix format : [ tn , fp , fn , tp ]

In [ ]:
print('Best Model By Accuracy')
print(models_eval.loc[models_eval['accuracy'] == max(models_eval['accuracy'])].model_name.to_string(index=False))
print('-----------------------')
print('Best Model By Recall')
print(models_eval.loc[models_eval['recall'] == max(models_eval['recall'])].model_name.to_string(index=False))
print('-----------------------')
print('Best Model By F1')
print(models_eval.loc[models_eval['f1_score'] == max(models_eval['f1_score'])].model_name.to_string(index=False))
print('-----------------------')
print('Best Model By ROC')
print(models_eval.loc[models_eval['roc_auc_score'] == max(models_eval['roc_auc_score'])].model_name.to_string(index=False))
print('-----------------------')

Best Model By Accuracy
gboost_clf
-----------------------
Best Model By Recall
svc_clf
ann_clf
-----------------------
Best Model By F1
gboost_clf
-----------------------
Best Model By ROC
svc_clf
-----------------------
time: 19.3 ms (started: 2023-01-16 13:15:37 +00:00)


In [ ]:
name = 'ensem_clf'

ensem_clf = VotingClassifier(estimators=[('m1', gboost_clf), ('m2', svc_clf), ('m3', xgboost_clf)],
                             voting = 'hard',
                             n_jobs = -1, 
                             ).fit(X_train,y_train)

y_true = y_val
y_pred = ensem_clf.predict(X_val)
evaluation_results = evaluation_metrics(y_true, y_pred)

models = models.append({'model_name': name, 
                        'model': ensem_clf, 
                        'parameters': ensem_clf.get_params()}, 
                       ignore_index=True)

models_eval = models_eval.append({'model_name': name, 
                                  'confusion_matrix' : evaluation_results[0], 
                                  'accuracy': evaluation_results[1], 
                                  'recall' : evaluation_results[2], 
                                  'f1_score': evaluation_results[3],
                                  'roc_auc_score': evaluation_results[4]}, 
                                 ignore_index=True)

# Model Validation Result

In [ ]:
display(models)

,model_name,model,parameters
0,log_clf,"LogisticRegression(C=0, max_iter=331, n_jobs=-...","{'C': 0, 'class_weight': None, 'dual': False, ..."
1,svc_clf,"SVC(C=8, class_weight='balanced', random_state...","{'C': 8, 'break_ties': False, 'cache_size': 20..."
2,bayes_clf,GaussianNB(var_smoothing=0.001),"{'priors': None, 'var_smoothing': 0.001}"
3,neigh_clf,"KNeighborsClassifier(algorithm='kd_tree', n_jo...","{'algorithm': 'kd_tree', 'leaf_size': 30, 'met..."
4,tree_clf,"DecisionTreeClassifier(criterion='entropy', ma...","{'ccp_alpha': 0.0, 'class_weight': None, 'crit..."
5,rnd_clf,"(DecisionTreeClassifier(max_depth=18, max_feat...","{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w..."
6,gboost_clf,([DecisionTreeRegressor(criterion='friedman_ms...,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'..."
7,xgboost_clf,"XGBClassifier(max_depth=8, min_child_weight=7,...","{'base_score': 0.5, 'booster': 'gbtree', 'cols..."
8,lightgbm_clf,"LGBMClassifier(learning_rate=0.05, max_depth=8...","{'boosting_type': 'gbdt', 'class_weight': None..."
9,ann_clf,<keras.engine.sequential.Sequential object at ...,[<keras.layers.core.dense.Dense object at 0x7f...


time: 182 ms (started: 2023-01-16 13:44:37 +00:00)


In [ ]:
display(models_eval)

,model_name,confusion_matrix,accuracy,recall,f1_score,roc_auc_score
0,log_clf,"[14046, 3022, 42, 80]",0.821757,0.655738,0.049628,0.739341
1,svc_clf,"[16317, 751, 14, 108]",0.955497,0.885246,0.220183,0.920623
2,bayes_clf,"[11475, 5593, 18, 104]",0.673589,0.852459,0.035745,0.762385
3,neigh_clf,"[16554, 514, 16, 106]",0.969168,0.868852,0.285714,0.919369
4,tree_clf,"[16930, 138, 24, 98]",0.990576,0.803279,0.547486,0.897597
5,rnd_clf,"[16838, 230, 21, 101]",0.985398,0.827869,0.445916,0.907197
6,gboost_clf,"[17017, 51, 20, 102]",0.995870,0.836066,0.741818,0.916539
7,xgboost_clf,"[17005, 63, 23, 99]",0.994997,0.811475,0.697183,0.903892
8,lightgbm_clf,"[16980, 88, 22, 100]",0.993601,0.819672,0.645161,0.907258
9,ann_clf,"[16079, 989, 14, 108]",0.941652,0.885246,0.177194,0.913651


time: 21.2 ms (started: 2023-01-16 13:44:37 +00:00)


In [ ]:
print('Best Model By Accuracy')
print(models_eval.loc[models_eval['accuracy'] == max(models_eval['accuracy'])].model_name.to_string(index=False))
print('-----------------------')
print('Best Model By Recall')
print(models_eval.loc[models_eval['recall'] == max(models_eval['recall'])].model_name.to_string(index=False))
print('-----------------------')
print('Best Model By F1')
print(models_eval.loc[models_eval['f1_score'] == max(models_eval['f1_score'])].model_name.to_string(index=False))
print('-----------------------')
print('Best Model By ROC')
print(models_eval.loc[models_eval['roc_auc_score'] == max(models_eval['roc_auc_score'])].model_name.to_string(index=False))
print('-----------------------')

Best Model By Accuracy
gboost_clf
-----------------------
Best Model By Recall
svc_clf
ann_clf
-----------------------
Best Model By F1
gboost_clf
-----------------------
Best Model By ROC
svc_clf
-----------------------
time: 23.5 ms (started: 2023-01-16 13:44:41 +00:00)


In [ ]:
models.to_csv('models_validation.csv')
!cp models_validation.csv "drive/My Drive/"

time: 202 ms (started: 2023-01-16 13:45:07 +00:00)


In [ ]:
models_eval.to_csv('models_validation_eval.csv')
!cp models_validation_eval.csv "drive/My Drive/"

time: 159 ms (started: 2023-01-16 13:45:09 +00:00)
